# Autoria

* Cristiano Sampaio Pinheiro, RA 256352
* Jhonatan Cléto, RA 256444
* Mylena Roberta dos Santos, RA 222687

# Bibliotecas e Paths

In [1]:
from sklearn.model_selection import GridSearchCV, GroupKFold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import pandas as pd

from google.colab import drive
drive.mount('/content/drive', force_remount = True)

PATH = '/content/drive/MyDrive/MC936/P4/'

Mounted at /content/drive


# Classificador SVM

De modo a tentar identificar a etiologia mais provável das lesões de substância branca presentes em pacientes com Lúpus Eritematoso Sistêmico (LES), optamos por utilizar um classificador SVM (*Support Vector Machine*). 

A fim de treinar o classificador e, por conseguinte, realizar a predição a partir dele, utilizamos *datasets* que contêm as *features* extraídas no *notebook* `image_processing.ipynb`.

Para o treinamento, empregamos os conjuntos de dados com as *features* extraídas das lesões de pacientes que sofreram Acidente Vascular Cerebral (AVC) ou têm Esclerose Múltipla (EM), visando que o classificador aprenda a diferenciá-las.

Para a predição, por sua vez, empregamos o conjunto de dados com as *features* extraídas das lesões de pacientes com LES.

## Preparação dos Datasets de Treino e Teste

Antes de propriamente treinar e utilizar o classificador para predizer a etiologia das lesões de pacientes com LES, foi necessário "preparar" minimamente os *datasets* citados para o uso.



### Dataset de Treino

No caso dos *datasets* empregados no treinamento, foi preciso concatenar as tabelas com os valores das *features* extraídas para as lesões de AVC e EM. Em seguida, optamos por remover as lesões em que alguma *feature* apresentava valor resultante NaN (*Not a Number*).

Ademais, separamos as colunas da tabela concatenada em três variáveis:

* `X_train (data)` - contém todos os resultados das *features* calculadas.
* `y_train (target)` - contém a doença que causou cada uma das lesões (AVC ou EM).
* `group_train` - contém os números dos pacientes associados a cada lesão.

Por fim, uniformizamos os dados contidos em `X_train` utilizando a classe [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler).

In [2]:
# Importação do DataFrame com as features de lesões de AVC
avc = pd.read_csv(PATH + '/data/avc_features.csv') 

# Importação do DataFrame com as features de lesões de EM
em = pd.read_csv(PATH + '/data/em_features.csv')

# Concatenação dos DataFrames
dataset = pd.concat([avc, em])

# Remoção dos registros com features NaNs
dataset = dataset.dropna().copy()
dataset.reset_index(drop = True, inplace = True)

X_train = dataset.iloc[:, 1:23]   # Data
y_train = dataset['target']       # Target
group_train = dataset['patient']  # N°s dos pacientes

# Uniformização dos valores das features
X_train = StandardScaler().fit_transform(X_train)

### Dataset de Teste

No caso do *dataset* empregado no teste, o processo foi semelhante, todavia mais simples. Inicialmente removemos as lesões em que alguma *feature* apresentava valor resultante NaN (*Not a Number*).

Ademais, separamos as colunas da tabela concatenada em duas variáveis:

* `X_test (data)` - contém todos os resultados das *features* calculadas.
* `group_test` - contém os números dos pacientes associados a cada lesão.

Por fim, uniformizamos os dados contidos em `X_test` utilizando a classe [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler).

In [3]:
# Importação do DataFrame com as features de lesões de LES
dataset = pd.read_csv(PATH + '/data/les_features.csv')

# Remoção dos registros com features NaNs
dataset = dataset.dropna().copy()
dataset.reset_index(drop = True, inplace = True)

X_test = dataset.iloc[:, 1:]     # Data
group_test = dataset['patient']  # N°s dos pacientes

# Uniformização dos valores das features
X_test = StandardScaler().fit_transform(X_test)

## Ajuste dos Hiperparâmetros do Classificador

Para definir os parâmetros do classificador, fizemos uso da classe [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV) com uma determinada gama parâmetros. Sendo que a escolha de tais parâmetros foi realizada considerando os conhecimentos obtidos através de pesquisas sobre o uso da classe em questão e a falta de compreensão profunda do domínimo do pergunta de pesquisa.

Empregamos a classe [GroupKFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GroupKFold) para fazer a validação cruzada pois, de acordo com a documentação, ela garante que o mesmo grupo (isto é, paciente) não esteja nos conjuntos de teste e treinamento. O número de *folds* foi determinado conforme as variações que testamos.

Ademais, "escolhemos" os melhores hiperparâmetros para o modelo com base no *score* de acurácia calculado para cada iteração de treinamento testada pelo GridSearchCV.

In [4]:
# Definição dos hiperparâmetros a serem testados
param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'degree': [1, 3, 5, 7, 9],
              'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}

# Definição da validação cruzada a ser utilizada
group_kfold = GroupKFold(n_splits = 10).split(X_train, y_train, groups = group_train)

# Definição dos parâmetros para a busca dos melhores hiperparâmetros
grid = GridSearchCV(estimator = SVC(), param_grid = param_grid, 
                    scoring = 'accuracy', n_jobs = 8, 
                    refit = True, cv = group_kfold, 
                    verbose = 0, return_train_score = True)

# Treino do modelo com os melhores hiperparâmetros
grid.fit(X_train, y_train, groups = group_train)

GridSearchCV(cv=<generator object _BaseKFold.split at 0x7f7251f0ced0>,
             estimator=SVC(), n_jobs=8,
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'degree': [1, 3, 5, 7, 9],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             return_train_score=True, scoring='accuracy')

In [5]:
C      = grid.best_params_['C']
gamma  = grid.best_params_['gamma']
degree = grid.best_params_['degree']
kernel = grid.best_params_['kernel']

query = f'(param_C == {C}) & (param_gamma == {gamma}) '
query = query + f'& (param_degree == {degree}) & (param_kernel == "{kernel}")'

# Impressão dos resultados do treinamento
print(pd.DataFrame(grid.cv_results_).query(query).T)

                                                                  106
mean_fit_time                                                0.170046
std_fit_time                                                 0.013571
mean_score_time                                              0.021677
std_score_time                                               0.006451
param_C                                                             1
param_degree                                                        1
param_gamma                                                       0.1
param_kernel                                                      rbf
params              {'C': 1, 'degree': 1, 'gamma': 0.1, 'kernel': ...
split0_test_score                                            0.966102
split1_test_score                                            0.913793
split2_test_score                                            0.965517
split3_test_score                                            0.948276
split4_test_score   

## Classificação do Dataset de Teste

Conforme o escopo do projeto, ao fim, usamos o classificador treinado para predizer a etiologia (isquêmica ou desmielinizante no caso, respectivamente, do resultado AVC ou EM) das lesões de substância branca dos pacientes com LES.

Realizada a predição, voltamos a associar cada resultado ao seu respectivo paciente e exportamos o *DataFrame* resultante em um arquivo CSV. De modo que esse arquivo será a base para a análise e compreensão dos resultados obtidos pelo classificador SVM.

In [6]:
# Predição das lesões do dataset de teste
y_pred = pd.Series(grid.predict(X_test))

# Concatenação dos n°s dos pacientes com a predição
df_pred = pd.concat([group_test, y_pred], axis = 'columns')
df_pred.columns = ['patient', 'target']

# Exportação da classificação das lesões em um arquivo CSV
df_pred.to_csv(PATH + '/data/les_classification.csv', index = False)